In [1]:
from features import process_features
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from xgboost import XGBClassifier

In [2]:
train = pd.read_csv('train.tsv', sep='\t')
train_id = pd.read_csv('train_id.tsv', sep='\t')
test = pd.read_csv('test_A.tsv', sep='\t')
test_id = test[['PERSONID']].drop_duplicates()

In [5]:
train_table = process_features(train, train_id)

Step1
Step2
Step3


In [6]:
test_table = process_features(test, test_id)

Step1
Step2
Step3


In [7]:
X = train_table.drop(['PERSONID', 'LABEL'], axis=1).values
Y = train_table['LABEL'].values
# X_test = test_table.drop('PERSONID', axis=1).values

In [8]:
X_test = test_table.drop('PERSONID', axis=1).values

In [9]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2)

In [15]:
def preprocess(X_train, X_val, X_test):
    vt = VarianceThreshold(threshold=0.05)
    poly = PolynomialFeatures(degree=1)
    X_train = vt.fit_transform(X_train)
    X_val = vt.transform(X_val)
    X_test = vt.transform(X_test)
    X_train = poly.fit_transform(X_train)
    X_val = poly.transform(X_val)
    X_test = poly.transform(X_test)
    return X_train, X_val, X_test

In [16]:
X_train_pre, X_val_pre ,X_test_pre = preprocess(X_train, X_val ,X_test)

In [24]:
xgb = XGBClassifier(max_depth =5,
                    n_estimators=1000,
                    booster='dart',
                    # colsample_bytree=0.8,
                    # subsample=0.2,
                    silent=False,
                    scale_pos_weight=3,
                    learning_rate=0.1)

In [25]:
xgb.fit(X_train_pre, Y_train,
       eval_set=[(X_val_pre, Y_val)],
       eval_metric='auc',
       early_stopping_rounds=10)

[0]	validation_0-auc:0.83625
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.842825
[2]	validation_0-auc:0.845337
[3]	validation_0-auc:0.845606
[4]	validation_0-auc:0.848524
[5]	validation_0-auc:0.854699
[6]	validation_0-auc:0.855027
[7]	validation_0-auc:0.857579
[8]	validation_0-auc:0.855938
[9]	validation_0-auc:0.861156
[10]	validation_0-auc:0.859169
[11]	validation_0-auc:0.860586
[12]	validation_0-auc:0.863886
[13]	validation_0-auc:0.867918
[14]	validation_0-auc:0.870619
[15]	validation_0-auc:0.871344
[16]	validation_0-auc:0.870822
[17]	validation_0-auc:0.870109
[18]	validation_0-auc:0.870275
[19]	validation_0-auc:0.871465
[20]	validation_0-auc:0.872076
[21]	validation_0-auc:0.872207
[22]	validation_0-auc:0.871647
[23]	validation_0-auc:0.873169
[24]	validation_0-auc:0.874233
[25]	validation_0-auc:0.874929
[26]	validation_0-auc:0.87551
[27]	validation_0-auc:0.875546
[28]	validation_0-auc:0.875106
[29]	validation_0-auc:0.875565
[30]	validation_0-

XGBClassifier(base_score=0.5, booster='dart', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=3, seed=None,
       silent=False, subsample=1)

In [26]:
Y_test_pred = xgb.predict_proba(X_test_pre)

In [27]:
Y_test_pred

array([[0.57647943, 0.4235206 ],
       [0.965639  , 0.034361  ],
       [0.9674135 , 0.03258651],
       ...,
       [0.69160295, 0.30839705],
       [0.9560891 , 0.04391091],
       [0.64386404, 0.35613596]], dtype=float32)

In [30]:
test_id['Pred'] = Y_test_pred[:,1]

In [32]:
test_id.to_csv('submission_test.csv', sep='\t', index=False, header=False)

In [34]:
Y_test_pred[:,1]

array([0.4235206 , 0.034361  , 0.03258651, ..., 0.30839705, 0.04391091,
       0.35613596], dtype=float32)

In [35]:
pd.read_csv('submission_test.csv', sep='\t')

,d7d27cd05dd10be17a9d5339bb1596ad,0.4235206
0,0500374e24dbff39d372acc0ca054bec,0.034361
1,8707556ccbb71387d58233d9aa6511f2,0.032587
2,f1e4dc175cb1970f6eb7f3b6f6cbc62c,0.029614
3,2c20d95ec8b23c5c634cb9468b74a265,0.067349
4,3eac2b9e136d4b18e0b1f350567b882f,0.024531
5,cd55526c5bacb2d582432efe85d07b42,0.021392
6,ba552bbf7ee8ed17a589b731f63dff0e,0.062451
7,f927a611cf21dfc7ebcde9c4d5199f1c,0.328209
8,07dad5f6ef5c6ba0cff77b26e0b996e4,0.035788
9,434d6feb79f28eacd0819a37a3c3d205,0.034248
